# Validation

In this part of the tutorial, we'll learn how to validate user responses.

Run the cell below to create a test app.

In [ ]:
import os

from hemlock import User, Page, create_test_app
from hemlock.functional import validate
from hemlock.questions import Check, Input, Label
from sqlalchemy_mutable.utils import partial

os.environ.pop("GITPOD_HOST", None)

app = create_test_app()

We've already seen examples of validation when we passed `input_tag={"type": "number", "min": 0}` to an `Input`. This requires the user, if they enter a response, to enter a number greater than 0.

Similarly, we can require users to respond to an `Input` by passing `input_tag={"required": True}`.

In [ ]:
def seed():
    return [
        Page(
            Input(
                "What's your name?",
                input_tag={"required": True}
            )
        ),
        Page(
            Label("Goodbye!")
        )
    ]

Unfortunately, we can't use the `input_tag={"required": True}` pattern for `Check` questions.

Instead, we'll use a *validate function*. Validate functions are similar to the compile functions we learned about before.

In this example, we'll use a built-in validate function `validate.require_response("My feedback message.")`. The feedback message is what the user will see if they don't respond.

Run the cell below and see what happens when the user doesn't respond.

In [ ]:
def seed():
    return [
        Page(
            Check(
                "Which of these animals do you like best?",
                ["Lions", "Tigers", "Bears"],
                validate=validate.require_response("Please select your favorite animal.")
            )
        ),
        Page(
            Label("Goodbye!")
        )
    ]


user = User.make_test_user(seed)
user.test_request([None]).display()

Now see what happens when the user *does* respond.

In [ ]:
user.test_request(["Lions"]).display()

We can use another built-in validate function, `validate.compare_response`, to compare the user's response to some value.

In the example below, we ask the user what's the most and least they would expect to pay for some item. We then validate that the least they would expect to pay is less than the most they would expect to pay.

Look at the expression 

```
validate.compare_response(
    max_payment,
    comparison="<=",
    feedback="The least you'd expect to pay should be less than the most you'd expect to pay."
)
```

This means, "The user's response to the current question should be less than or equal to their response to `max_payment`. If it isn't, tell them, 'The least you'd expect to pay should be less than the most you'd expect to pay.'"

Run the cell below to see what happens when the user enters an invalid response.

In [ ]:
def seed():
    return [
        Page(
            max_payment := Input(
                "What's the most you would expect to pay for this item?",
                input_tag={"type": "number", "min": 0, "required": True}
            ),
            Input(
                "What's the least you would expect to pay for this item?",
                input_tag={"type": "number", "min": 0, "required": True},
                # this is the validate function referenced above
                validate=validate.compare_response(
                    max_payment,
                    comparison="<=",
                    feedback="The least you'd expect to pay should be less than the most you'd expect to pay."
                )
            )
        ),
        Page(
            Label("Goodbye!")
        )
    ]


user = User.make_test_user(seed)
user.test_request([10, 20]).display()

Run the cell below to see what happens when the user enters a valid response.

In [ ]:
user.test_request([10, 2]).display()

We can also define our own validate functions.

In the example below, we ask the user to imagine they have a $50 gift card to spend at a clothing store. We validate that the amount they spend on shirts and pants is equal to the gift card amount.

If the user's response was invalid, the validate function should return either:

- `False`
- `(False, "Some feedback message.")`

If the user's response was valid, the validate function doesn't have to return anything.

Note that we require the user to respond to both the shirts spending and pants spending input using `input_tag={"required": True}`. This is because, if the user doesn't respond to either of these questions (that is, their responses are `None`), our validate function will fail when it tries to evaluate `pants_spending.response + shirts_spending.response`.

Run the cell below and see what happens when the user's total spending doesn't equal the gift card amount.

In [ ]:
GIFT_CARD_VALUE = 50

def seed():
    return [
        Page(
            Label(f"Imagine you have a ${GIFT_CARD_VALUE} gift card for a clothing store."),
            shirts_spending := Input(
                "How much would you spend on shirts?",
                input_tag={"type": "number", "min": 0, "required": True}
            ),
            Input(
                "How much would you spend on pants?",
                input_tag={"type": "number", "min": 0, "required": True},
                validate=partial(validate_total_spending, shirts_spending)
            )
        ),
        Page(
            Label("Goodbye!")
        )
    ]


def validate_total_spending(pants_spending, shirts_spending):
    if pants_spending.response + shirts_spending.response != GIFT_CARD_VALUE:
        return False, f"The total amount you spend on shirts and pants should be ${GIFT_CARD_VALUE}."


user = User.make_test_user(seed)
user.test_request([None, 20, 20]).display()

Run the cell below to see what happens when the user's response is valid.

In [ ]:
user.test_request([None, 20, 30]).display()

## Exercises

0. Create a survey with 3 pages:

    0. A page with two questions:
    
        0. An "original ID" `Input` asking the user to enter an ID (e.g., "What's your MTurk ID?").
        1. A "confirmation" `Input` (e.g., "Please enter your MTurk ID a second time.")
    1. A page with two questions:

        0. A `Check` question asking if the user made a new year's resolution to lose weight.
        1. An `Input` asking, if so, how many pounds they want to lose.
    2. A goodbye page.
1. Add validation requiring the user to respond to the original ID `Input` on page 0.
2. Add validation requiring the user's original ID and confirmation to match.
3. Add validation requiring the user to enter the amount of weight they want to lose (the `Input` on page 1) if they made a new year's resolution to lose weight. Hint: Use a custom validation function, not `input_tag={"required": True}`.
4. Make a test user and have it do the following:

    0. Try to go forward from page 0 when the original ID doesn't match the confirmation.
    1. Successfully go forward from page 0 (the original ID should match the confirmation).
    2. Try to go forward from page 1 when the user resolved to lose weight but didn't enter the amount of weight they want to lose.
    3. Successfully go forward from page 1. You can either do this by:
    
        0. Entering the amount of weight the user wants to lose, or
        1. Saying that the user didn't resolve to lose weight
5. Transfer the seed function you wrote in steps 0-4 to `src/my_survey.py`, run the app, and repeat the exercise from step 5 in your browser. Additionally, see what happens when you try to go forward from the 0th page without entering your ID.
6. Test your code with `make test`. Note that you'll have to set the test user's default response to the ID inputs on page 0 to make sure they match. Otherwise, your test users will get stuck.

In [ ]:
# WRITE YOUR CODE HERE

## Answers

In [ ]:
def seed():
    return [
        Page(
            id_input := Input(
                "What is your MTurk ID?",
                input_tag={"required": True}
            ),
            Input(
                "Confirm your MTurk ID",
                validate=validate.compare_response(
                    id_input, feedback="MTurk IDs do not match."
                )
            )
        ),
        Page(
            resolved_to_lose_weight := Check(
                "Did you make a new year's resolution to lose weight?",
                [(1, "Yes"), (0, "No")]
            ),
            Input(
                "If so, how many pounds do you want to lose?",
                input_tag={"type": "number", "min": 0},
                validate=partial(require_if, resolved_to_lose_weight)
            )
        ),
        Page(
            Label("Goodbye!")
        )
    ]


def require_if(amount_input, resolved_to_lose_weight):
    if resolved_to_lose_weight.response and amount_input.response is None:
        return False, "Please enter how many pounds you want to lose."


user = User.make_test_user(seed)
user.test_request(["some_id", "some_other_id"]).display()

In [ ]:
user.test_request(["some_id", "some_id"]).display()

In [ ]:
user.test_request([1, None]).display()

In [ ]:
user.test_request([1, 10]).display()

In [ ]:
user = User.make_test_user(seed)
user.test_request(["some_id", "some_id"])
user.test_request([0, None]).display()

See `src/validate.py` for what your survey file should look like.

Now you know how to add validation! Check out `080_submit.ipynb` for the next part of the tutorial.